In [3]:
import pandas as pd
from datetime import datetime, timedelta

import yfinance as yf
import requests
import time

In [17]:
# Yahoo Finance
yf_tickers = ['AAPL', 'GOOGL', 'MSFT', 'META']
start_date = '2025-01-01'
end_date = '2025-06-23'

def get_yf_prices(tickers, start, end):
    data = yf.download(tickers, start=start, end=end, group_by='ticker', auto_adjust=True, progress=False)
    daily_prices = []
    for ticker in tickers:
        df = data[ticker].reset_index()[['Date', 'Close']]
        df['asset_ticker'] = ticker
        df.rename(columns={'Date': 'date', 'Close': 'price_usd'}, inplace=True)
        daily_prices.append(df)
    return pd.concat(daily_prices, ignore_index=True)

yf_prices_df = get_yf_prices(yf_tickers, start_date, end_date)

In [14]:
# CoinGecko
def get_coingecko_prices(coin_id, days=365):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
    params = {'vs_currency': 'usd', 'days': days, 'interval': 'daily'}
    r = requests.get(url, params=params)
    data = r.json()['prices']
    df = pd.DataFrame(data, columns=['timestamp', 'price_usd'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['asset_ticker'] = coin_id.upper()
    return df[['date', 'asset_ticker', 'price_usd']]

cg_cryptos = ['bitcoin', 'ethereum']
cg_dfs = []
for coin in cg_cryptos:
    cg_dfs.append(get_coingecko_prices(coin))
    time.sleep(1)

crypto_prices_df = pd.concat(cg_dfs)
crypto_prices_df['asset_ticker'] = crypto_prices_df['asset_ticker'].replace({'BITCOIN': 'BTC', 'ETHEREUM': 'ETH'})